In [ ]:
import cartopy.crs as ccrs
from cmocean import cm 
from dino import Experiment
from matplotlib import colors
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import cftime as cft
import xnemogcm as xn
import xgcm

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
dino_exp = Experiment(path, 'HigherRidgeEmP/restart10')

In [ ]:
import scipy.sparse as sparse
import scipy.sparse.linalg as la

def _get_dynmodes(Nsq, e3t, e3w, nmodes=2):
    """
    Calculate the 1st nmodes ocean dynamic vertical modes.
    Based on
    http://woodshole.er.usgs.gov/operations/sea-mat/klinck-html/dynmodes.html
    by John Klinck, 1999.
    """
    # 2nd derivative matrix plus boundary conditions
    Ndz     = (Nsq * e3w)
    e3t     = e3t
    #Ndz_m1  = np.roll(Ndz, -1)
    #e3t_p1  = np.roll(e3t, 1)
    d0  = np.r_[1. / Ndz[1] / e3t[0],
               (1. / Ndz[2:-1] + 1. / Ndz[1:-2]) / e3t[1:-2],
               1. / Ndz[-2] / e3t[-2]]
    d1  = np.r_[0., -1. / Ndz[1:-1] / e3t[1:-1]]
    dm1 = np.r_[-1. / Ndz[1:-1] / e3t[0:-2], 0.]
    diags = 1e-4 * np.vstack((d0, d1, dm1))
    d2dz2 = sparse.dia_matrix((diags, (0, 1, -1)), shape=(len(Nsq)-1, len(Nsq)-1))
    # Solve generalized eigenvalue problem for eigenvalues and vertical
    # Horizontal velocity modes
    try:
        eigenvalues, modes = la.eigs(d2dz2, k=nmodes+1, which='SM')
        mask = (eigenvalues.imag == 0) & (eigenvalues >= 1e-10)
        eigenvalues = eigenvalues[mask]
        # Sort eigenvalues and modes and truncate to number of modes requests
        index = np.argsort(eigenvalues)
        eigenvalues = eigenvalues[index[:nmodes]].real
        # Modal speeds
        ce = 1 / np.sqrt(eigenvalues * 1e4)
    except:
        ce = -np.ones(nmodes)

    return(ce)

In [ ]:
import scipy.sparse as sp
import scipy.sparse.linalg as la
from scipy.linalg import eig

def compute_vmodes_1D(Nsqr, dzc=None, dzf=None, nmodes=2): 
    """
    Compute vertical modes from stratification. Assume grid is sorted downoward (first point at surface, last point at bottom) and depth is algebraic (i.e. negative)
    Take either vertical grid metrics (spacing) or levels as inputs. 
    Need 2 staggered grid (center and left or outer), with Nsqr specified on left/outer grid
    No normalization. Pressure mode is positive at the surface.

    Parameters:
    ___________
    N2f: (N,) ndarray
        Brunt-Vaisala frequency at cell left points
    dzc: (N) ndarray, optional
        vertical grid spacing at cell centers. Either dzc, dzf or zc, zf must be passed
    dzf: (N) ndarray
        vertical grid spacing at cell left points
    nmodes: int, optional
        number of baroclinic modes to compute (barotropic mode will be added)

    Returns:
    ________
    c: (nmodes) ndarray
        eigenvalues (pseudo phase speed, c=1/sqrt(k))
    !! Currently not returning the modes since they are not needed
    phi: (N,nmodes) ndarray
        p-like modes at cell centers
    phiw: (N,nmodes) ndarray
        w-like modes at cell interfaces. phiw' = phi

    Notes:
    ______
    The vertical modes are definied following the equation:
    .. math:: (\phi'/N^2)' + k^2\phi=0 
    with boundary condition :math:`\phi'=0` at the bottom and :math:`g\phi' + N^2\phi=0` at the surface (or :math:`\phi'=0` for a rigid lid condition). 
    Computation of the vertical modes is performed using second order finite difference with staggered grid

    """
    ### parameters:
    g = 9.80665

    ### deal with vertical grids
    Nz = Nsqr.size
    if dzc is not None and dzf is not None:
        dz_surf = .25*(dzc[0] + dzf[0]) ### this is approx for NEMO grid
        dzc, dzf = dzc, dzf
    else:
        raise ValueError("must specify grid increments dzc, dzf") 

    invg = np.ones(1)/g
    
    Nsqog = Nsqr[:1]*invg

    v12 =  np.stack([1./np.r_[dzc, np.ones(1),], -1./np.r_[np.ones(1), dzc]])
    Dw2p = sp.spdiags(v12,[0, 1],Nz,Nz,format="lil")
    ### vertical derivative matrix, p-to-w grids, targetting inner w points only
    v12 =  np.stack([1./np.r_[dzf[1:], np.ones(1)], -1./dzf])
    Dp2w = sp.spdiags(v12,[-1, 0],Nz,Nz,format="lil")
    
    ### second order diff matrix
    D2z = Dw2p*Dp2w
    Dp2w[0,0] = -Nsqog*(1-Nsqog*dz_surf) # surface boundary condition (free or rigid lid)
    ### formulation of the problem : -dz(dz(p)/N^2) = lambda * p
    A = - Dw2p * sp.diags(1./Nsqr) * Dp2w
#    return A
    # compute numerical solution
    ev,ef = la.eigs(A.tocsc(), k=nmodes+1, which='SM')

    ## select and arrange modes
    inds = np.isfinite(ev)
    ev, ef = ev[inds].real, ef[:,inds].real
    isort = np.argsort(ev)[:nmodes+1]
    ev, ef = ev[isort], ef[:,isort]
    # ef *= np.sign(ef[0,:])[None,:] # positive pressure at the surface
    # if first_ord:
        # pmod, wmod = ef[:Nz,:], -ef[Nz:,:]
    # else:
        # pmod = ef[:Nz,:]
        # wmod = -(Dp2w * pmod) / (Nsqr[:,None] * ev[None,:])
        # if not (free_surf and g>0):
            # wmod[:,0] = 0.
    
    return 1./ev**.5 #, pmod, wmod

In [ ]:
def get_vmodes(exp, nmodes=2):
    """ compute vertical modes
    Wrapper for calling `compute_vmodes` with DataArrays through apply_ufunc. 
    z levels must be in descending order (first element is at surface, last element is at bottom) with algebraic depth (i.e. negative)
    Normalization is performed here (int_z \phi^2 \dz = Hbot)
    
    Parameters:
    ___________
    ds: xarray.Dataset
        contains brunt-vaisala frequency and vertical grid information (levels of metrics, i.e. spacing)
    nmodes: int, optional
        number of vertical baroclinic modes (barotropic is added)
    
    Returns:
    ________
    xarray.DataSet: vertical modes (p and w) and eigenvalues
    !! (currently only eigenvalues)
    _________
    """
    Nsq = (exp.get_N_squared())
    res = xr.apply_ufunc(_get_dynmodes, 
                         Nsq.isel(x_c=slice(1,-1), y_c=slice(1,-1), t_y=-1).chunk({'z_f':-1}),
                         exp.data.e3t.where(exp.domain.tmask==1.0).isel(x_c=slice(1,-1), y_c=slice(1,-1), t_y=-1).chunk({'z_c':-1}),
                         exp.data.e3w.isel(x_c=slice(1,-1), y_c=slice(1,-1), t_y=-1).chunk({'z_f':-1}),
                         input_core_dims=[['z_f'],['z_c'],['z_f']],
                         dask='parallelized', vectorize=True,
                         output_dtypes=[Nsq.dtype],
                         output_core_dims=[["mode"]],
                         dask_gufunc_kwargs={"output_sizes":{"mode":nmodes}}
                        )
    # res['mode'] = np.arange(nmodes+1)
    # # unstack variables
    # c = res.isel(s_stack=0)
    # phi = (res.isel(s_stack=slice(1,N+1))
    #        .rename('phi')
    #        .rename({'s_stack': zc})
    #        #.assign_coords(z_rho=zc)
    #       )
    # if "z_del" in kwargs:
    #     dzc = ds[kwargs["z_del"]["zc"]]
    # else:
    #     dzc = ds["e3t"] # use default value for NEMO    
    # norm_tg = dzc.where(ds.tmask).sum(zc)
    # norm = (phi**2*dzc).where(ds.tmask).sum(zc) 
    # phi /= (norm/norm_tg)**.5 # 1/H \int(phi^2 dz) = 1
    # phiw = (res.isel(s_stack=slice(N+1,2*N+1))
    #           .rename('phiw')
    #           .rename({'s_stack': zl})
    #         #  .assign_coords(z_w=zf)
    #          ) / (norm/norm_tg)**.5
    # norm = norm_tg # norm = int(phi^2 dz)
    # # merge data into a single dataset
    # dm = xr.merge([c.rename("c"), phi.rename("phi"), 
    #                phiw.rename("phiw"), norm.rename("norm")
    #              ])
    return res  ### hard-coded norm = H 

In [ ]:
ce, modes = _get_dynmodes(
    dino_exp.get_N_squared().isel(t_y=-1, y_c=100, x_c=30).values,
    dino_exp.data.e3t.isel(t_y=-1, y_c=100, x_c=30).values,
    dino_exp.data.e3w.isel(t_y=-1, y_c=100, x_c=30).values
)


In [ ]:
plt.plot(dino_exp.domain.gdept_1d.values[:-1],modes[:,0].real)
plt.plot(dino_exp.domain.gdept_1d.values[:-1],modes[:,1].real)
plt.plot(dino_exp.domain.gdept_1d.values[:-1],modes[:,2].real)

In [ ]:
ce = get_vmodes(dino_exp, nmodes=2)

In [ ]:
ce = ce.compute()

In [ ]:
ce_inter_x = ce.where(ce>=0).interpolate_na('x_c')
ce_inter_y = ce.where(ce>=0).interpolate_na('y_c')

In [ ]:
ce_inter = (ce_inter_x + ce_inter_y) / 2.

In [ ]:
ce_inter.isel(mode=1).plot()

In [ ]:
f = 2 * 7.2921e-5 * np.sin(2 * np.pi * dino_exp.domain.gphit / 360)

In [ ]:
beta_f = dino_exp.grid.derivative(f, 'Y')
beta_t = dino_exp.grid.interp(beta_f, 'Y')

In [ ]:
L = np.sqrt(ce_inter**2 / (f.isel(x_c=slice(1,-1), y_c=slice(1,-1))**2 + 2 * beta_t.isel(x_c=slice(1,-1), y_c=slice(1,-1)) * ce_inter))

In [ ]:
res = (L.isel(mode=1) / 2 / dino_exp.domain.e2t).assign_coords({'x_globe': L.glamt - 29})
#res = (L.isel() / 2 / dino_exp.domain.e2t).assign_coords({'x_globe': L.glamt - 29})

fig = plt.figure(figsize=(10,10))
p = res.plot.contourf(
    y='gphit',
    x='x_globe',
    cmap=cm.thermal_r,
    levels=np.array([1, 1/2, 1/4, 1/8, 1/16, 1/32, 1/64]),
    subplot_kws=dict(projection=ccrs.Robinson()),

    transform=ccrs.PlateCarree(),
    add_colorbar=False
)
p.axes.gridlines(
    draw_labels=["y", "left", "top", "geo"],
    ylocs=[-70, -45, -20, 0, 20, 45, 70],
    xlocs=[-29, 0, 29]
)
cbar1 = fig.colorbar(p, label=r'$\psi$ [Sv]', ticks=[1, 1/2, 1/4, 1/8, 1/16, 1/32, 1/64])
cbar1.ax.set_yticklabels(['1°', '1/2°', '1/4°', '1/8°', '1/16°', '1/32°', '1/64°'])
plt.title('')
plt.grid('m')
plt.tight_layout()
#plt.savefig('DINO_bathy_3.png', facecolor=(1,1,1,0))

In [ ]:
dino_exp.get_N_squared().isel(t_y=-1, x_c=30).plot(norm=colors.LogNorm(), yincrease=False)